In [1]:
!pip install tensorflow_addons

  Using cached tensorflow_addons-0.21.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (612 kB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)


In [2]:
#Imports and settings

import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import sys

import tensorflow as tf
from tqdm.notebook import tqdm
import tensorflow_addons as tfa


from sklearn.model_selection import train_test_split

np.set_printoptions(suppress=True)

/opt/conda/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/opt/conda/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're usin

In [3]:
# load the features and labels (subtract 1 as the labels aren't indexed from 0)
ytest = np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/test/y_test.txt')-1
ytrain = np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/train/y_train.txt')-1

In [4]:
# load the x,y,z body accelerations test data
xx_test=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/test/Inertial Signals/body_acc_x_test.txt')
yy_test=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/test/Inertial Signals/body_acc_y_test.txt')
zz_test=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/test/Inertial Signals/body_acc_z_test.txt')
# concatenate the arrays along the last dimension
xtest = np.concatenate((xx_test[:,:,None],yy_test[:,:,None],zz_test[:,:,None]),axis=2)
# (using None here adds an extra dimension of size 1 to the end of the array)

In [5]:
# follow the same approach for the train data
xx_train=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/train/Inertial Signals/body_acc_x_train.txt')
yy_train=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/train/Inertial Signals/body_acc_y_train.txt')
zz_train=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/train/Inertial Signals/body_acc_z_train.txt')
xtrain = np.concatenate((xx_train[:,:,None],yy_train[:,:,None],zz_train[:,:,None]),axis=2)

In [6]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(7352, 128, 3)
(2947, 128, 3)
(7352,)
(2947,)


## Part 2
### 1‐dimensional convolutional neural network
Note: Your training data tensors will need to have their original shape of [n,128,3] so make sure to recreate the tensors after
you have completed part 1

In [7]:
#tensorboad set up
logdir = '/home/jovyan/logs/Project/Part2'

In [8]:
#Recreate the tensors
x_input, x_val, y_input, y_val = train_test_split(xtrain, ytrain, test_size=0.2, random_state=10)

#one-hot encoding for labels
y_input = tf.keras.utils.to_categorical(y_input)
y_val = tf.keras.utils.to_categorical(y_val)

print(xtrain.shape)
print(xtest.shape)
print(y_input.shape)
print(y_val.shape)

(7352, 128, 3)
(2947, 128, 3)
(5881, 6)
(1471, 6)


In [9]:
# Convert your input and validation data to a tensor so that you can pass it into your classifier
x_input = tf.convert_to_tensor(x_input, tf.float32)
x_val = tf.convert_to_tensor(x_val, tf.float32)
y_input = tf.convert_to_tensor(y_input, tf.float32)
y_val = tf.convert_to_tensor(y_val, tf.float32)

2023-07-15 15:42:17.175275: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-15 15:42:17.708072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14660 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0001:00:00.0, compute capability: 7.5


In [10]:
# Keras API model

#We create the input layer and combine with further dense layers,
input_layer = tf.keras.Input(shape=(128, 3))
output_layer = tf.keras.layers.Conv1D(filters=128, kernel_size=16)(input_layer)
output_layer = tf.keras.layers.BatchNormalization()(output_layer)
output_layer = tf.keras.layers.Dense(64, activation='relu')(output_layer)
output_layer = tf.keras.layers.GlobalAveragePooling1D()(output_layer)

# # Add a softmax layer with  output units:
output_layer = tf.keras.layers.Dense(6, activation='softmax')(output_layer)


In [11]:
# Compile model

# Callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# create a callback that will stop training if the validation loss hasn't improved for 2 epochs
early_stopping_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)]
# initialize tqdm callback with default parameters
tqdm_callback = tfa.callbacks.TQDMProgressBar()

model = tf.keras.Model(inputs=input_layer,outputs=output_layer)

#The compilation and training then proceeds as per the sequential model,
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 3)]          0         
                                                                 
 conv1d (Conv1D)             (None, 113, 128)          6272      
                                                                 
 batch_normalization (BatchN  (None, 113, 128)         512       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 113, 64)           8256      
                                                                 
 global_average_pooling1d (G  (None, 64)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_1 (Dense)             (None, 6)                 390   

In [12]:
model.fit(x_input, y_input, 
          epochs=200, 
          batch_size=128,
          validation_data=(x_val, y_val),
          callbacks=[tensorboard_callback, tqdm_callback]
         )

Training:   0%|           0/200 ETA: ?s,  ?epochs/s

Epoch 1/200


0/46           ETA: ?s - 

Epoch 1/200


2023-07-15 15:42:22.415817: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


...

Epoch 198/200
46/46 [==============================] - 0s 4ms/step - loss: 0.3069 - accuracy: 0.8738 - val_loss: 0.2927 - val_accuracy: 0.8912
Epoch 199/200


0/46           ETA: ?s - 

Epoch 199/200
46/46 [==============================] - 0s 4ms/step - loss: 0.3057 - accuracy: 0.8692 - val_loss: 0.3022 - val_accuracy: 0.8838
Epoch 200/200


0/46           ETA: ?s - 

Epoch 200/200
46/46 [==============================] - 0s 4ms/step - loss: 0.3008 - accuracy: 0.8784 - val_loss: 0.3012 - val_accuracy: 0.8912


In [ ]:
predictions = model.predict(x_input,verbose=0)      ##TO DO: CHECK
y_pred = np.argmax(predictions,axis=-1)
print('train accuracy: ', np.sum(y_pred==y_input[:,0])/len(y_input))

In [ ]:
predictions = model.predict(x_val,verbose=0)
y_pred = np.argmax(predictions,axis=-1)
print('val accuracy: ', np.sum(y_pred==y_val[:,0])/len(y_val))

In [13]:
#one-hot encoding for labels
xtest = tf.convert_to_tensor(xtest, tf.float32)
ytest = tf.keras.utils.to_categorical(ytest)
print(xtest.shape)
print(ytest.shape)
loss, accuracy = model.evaluate(xtest, ytest, verbose = 1)
print("Test Accuracy :", accuracy)
print("Test Loss :", loss)

(2947, 128, 3)
(2947, 6)
93/93 [==============================] - 0s 2ms/step - loss: 0.6062 - accuracy: 0.8497
Test Accuracy : 0.8496776223182678
Test Loss : 0.606227695941925


## Other models

In [ ]:
#Recreate the tensors
x_input, x_val, y_input, y_val = train_test_split(xtrain, ytrain, test_size=0.2, random_state=10)

#one-hot encoding for labels
y_input = tf.keras.utils.to_categorical(y_input)
y_val = tf.keras.utils.to_categorical(y_val)
# Convert your input and validation data to a tensor so that you can pass it into your classifier
x_input = tf.convert_to_tensor(x_input, tf.float32)
x_val = tf.convert_to_tensor(x_val, tf.float32)
y_input = tf.convert_to_tensor(y_input, tf.float32)
y_val = tf.convert_to_tensor(y_val, tf.float32)

In [ ]:
##LSTM##
#Adapting
# https://www.analyticsvidhya.com/blog/2021/07/implementing-lstm-for-human-activity-recognition-using-smartphone-accelerometer-data/
ytrain = tf.keras.utils.to_categorical(ytrain)




from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout

model = Sequential()
# RNN layer
model.add(LSTM(units = 128, input_shape = (x_input.shape[1], x_input.shape[2])))
# Dropout layer
model.add(Dropout(0.5)) 
# Dense layer with ReLu
model.add(Dense(units = 124, activation='relu'))
# Softmax layer
model.add(Dense(y_input.shape[1], activation = 'softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
n_epochs = 200
batch_size = 1024 

history = model.fit(xtrain, ytrain, epochs = n_epochs, validation_split = 0.20, batch_size = batch_size, verbose = 1)

plt.plot(np.array(history.history['loss']), "r--", label = "Train loss")
plt.plot(np.array(history.history['accuracy']), "g--", label = "Train accuracy")
plt.plot(np.array(history.history['val_loss']), "r-", label = "Validation loss")
plt.plot(np.array(history.history['val_accuracy']), "g-", label = "Validation accuracy")
plt.title("Training session's progress over iterations")
plt.legend(loc='lower left')
plt.ylabel('Training Progress (Loss/Accuracy)')
plt.xlabel('Training Epoch')
plt.ylim(0) 
plt.show()